In [88]:
from collections import Counter

In [82]:
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    truthful = t.read()
    deceptive =d.read()

In [85]:
truthful_unigram = dict(Counter(truthful.split()))
deceptive_unigram = dict(Counter(deceptive.split()))

In [79]:
def get_unigram_counts(words):
    corpus = {}
    for word in words.split():
        if word not in corpus:
            corpus[word] = 1
        else:
            corpus[word] += 1
    return corpus